### What is AutoGrad?
- This is automatic Differentiation Engine that helps the neural network to optimize its performance.
- `torch.autograd()` provides Classes and functions implementing automatic differentiation of arbitary scaler value function.
- To compute the gradient we are using `.backward()` function that form acyclic graph, which stores the history of computation.

In [1]:
import torch

# Tensor wihtout gradient
x = torch.rand(3,3)
print("3x3 Tensor:\n", x)

3x3 Tensor:
 tensor([[0.1670, 0.8011, 0.6163],
        [0.0406, 0.3943, 0.6229],
        [0.4281, 0.6103, 0.2381]])


- `required_grad=True` track all the operation on the tensor will be tracked.
- we will get benefits and usage in the later.

In [2]:
# Tensor with gradient
x = torch.rand(3,3, requires_grad=True)
print(x)

tensor([[0.4968, 0.1123, 0.9465],
        [0.2396, 0.6592, 0.2948],
        [0.4272, 0.5338, 0.1749]], requires_grad=True)


In [3]:
y = x + 5
print("y Tensor after adding Grad True with x:\n", y)

y Tensor after adding Grad True with x:
 tensor([[5.4968, 5.1123, 5.9465],
        [5.2396, 5.6592, 5.2948],
        [5.4272, 5.5338, 5.1749]], grad_fn=<AddBackward0>)


In [4]:
print(y.grad_fn)

In [5]:
print(y.requires_grad)

True


In [6]:
# tracking a multiplicative gradient
x = torch.rand(3,3, requires_grad=True)
y = x * 2
print(y) # automatically operation are tracked
print(y.grad_fn)

# Confirm requires_grad
print(y.requires_grad)

tensor([[1.8927, 1.9646, 1.2404],
        [0.5084, 1.5763, 1.8005],
        [1.6676, 1.8217, 1.4911]], grad_fn=<MulBackward0>)
True


In [17]:
x = torch.ones(2,2, requires_grad=True)
y = x + 3
z = y ** 2
res = z.mean()
print("Z:\n",z)
print("Result:\n", res)

Z:
 tensor([[16., 16.],
        [16., 16.]], grad_fn=<PowBackward0>)
Result:
 tensor(16., grad_fn=<MeanBackward0>)


- Two important method `.backward()` => used for backward propogation and `.grad()` used for calculate the gradient.

In [18]:
res.backward()
print(x.grad)

tensor([[2., 2.],
        [2., 2.]])


### Computational Graph
- **Leaf Nodes** : A tensor is leaf tensor if it has `required_grad = False`
- We populate the gradients of a leaf tensor only using `backward()` and `requires_grad` for that tensor should be `True`. Even if a tensor has `requires_grad=True` but it is created by a user, then also it is a leaf tensor by default.

So, to sum it up:
1. We can use `backward()` on a tensor, if it is created because of operations performed on tensors which have `requires_grad=True`.
2. A leaf node with no `grad_fn` cannot have the gradients populated backward.
3. To populate the gradients, we will need `grad_fn` and the tensor should be a leaf tensor.

First, let’s try some tensor operations with requires_grad=False.

In [19]:
x = torch.ones(1,1)
print(x.requires_grad, x.grad_fn, x.is_leaf)
y = torch.ones(1,1)
print(y.requires_grad, y.grad_fn, y.is_leaf)
z = x + y
print(z)
print(z.requires_grad, z.grad_fn, z.is_leaf)

False None True
False None True
tensor([[2.]])
False None True


In [22]:
# z.backward() # it will give error like below has

```python
RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn
---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
~\AppData\Local\Temp/ipykernel_12056/148511205.py in <module>
----> 1 z.backward()

~\miniconda3\envs\pytorch19\lib\site-packages\torch\_tensor.py in backward(self, gradient, retain_graph, create_graph, inputs)
    253                 create_graph=create_graph,
    254                 inputs=inputs)
--> 255         torch.autograd.backward(self, gradient, retain_graph, create_graph, inputs=inputs)
    256 
    257     def register_hook(self, hook):

~\miniconda3\envs\pytorch19\lib\site-packages\torch\autograd\__init__.py in backward(tensors, grad_tensors, retain_graph, create_graph, grad_variables, inputs)
    145         retain_graph = create_graph
    146 
--> 147     Variable._execution_engine.run_backward(
    148         tensors, grad_tensors_, retain_graph, create_graph, inputs,
    149         allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn
```

![](https://debuggercafe.com/wp-content/uploads/2019/11/no_grad.png)

`required_grad = True` approach

In [23]:
x = torch.ones(1,1, requires_grad=True)
print(x.requires_grad, x.grad_fn, x.is_leaf)
y = torch.ones(1,1, requires_grad=True)
print(y.requires_grad, y.grad_fn, y.is_leaf)
z = x + y
print(z) 
print(z.requires_grad, z.grad_fn, z.is_leaf)
z.backward()
print(z)

True None True
True None True
tensor([[2.]], grad_fn=<AddBackward0>)
True <AddBackward0 object at 0x000001CB13AD9340> False
tensor([[2.]], grad_fn=<AddBackward0>)


![](https://debuggercafe.com/wp-content/uploads/2019/11/grad_fn.png)